In [1]:
import sys
sys.path.append('../')
from utils import preprocess

import numpy as np
import h5py

import pandas as pd

from collections import defaultdict
import string

from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split, KFold, GridSearchCV

import keras
from keras.layers import Dense, GlobalAveragePooling1D, Embedding, Lambda
import keras.backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

np.random.seed(1234)

Using TensorFlow backend.


In [2]:
def create_docs(df, n_gram_max=1):
    docs = []
    for i, text in enumerate(df.text):    
        def add_ngram(q, n_gram_max):
            ngrams = []
            for n in range(2, n_gram_max+1):
                for w_index in range(len(q)-n+1):
                    ngrams.append('--'.join(q[w_index:w_index+n]))
            return q + ngrams

        doc = preprocess(text).split()
        docs.append(' '.join(add_ngram(doc, n_gram_max)))
        
    return docs

In [3]:
df = pd.read_csv('./../data/train_feature.csv')
df_test = pd.read_csv('./../data/test_feature.csv')
text = df.text.values
text_test = df_test.text.values

author2class = {'EAP': 0, 'HPL' : 1, 'MWS' : 2}
class2author = ['EAP', 'HPL', 'MWS']
y = np.array([author2class[a] for a in df.author])
y = to_categorical(y)

In [4]:
def create_model(input_dim, embeddings_dims=20):
    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=embedding_dims))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [5]:
n_gram_max = 1
embedding_dims = 10

raw_docs = create_docs(df, n_gram_max=n_gram_max)
raw_docs_test = create_docs(df_test, n_gram_max=n_gram_max)

seed = 7
num_split = 5
epochs = 50

# for next training
predict_prob_features = np.zeros((len(df), 3))
predict_prob_features_test = np.zeros((len(df_test), 3))

ite = 0
sum_loss = 0.
min_count = 1

kf = KFold(n_splits=num_split, random_state=seed, shuffle=True)
for train_index, val_index in kf.split(text):
    ite += 1
    print('{}/{}: #Trains: {}, #Val: {}'.format(ite, num_split, len(train_index), len(val_index)), end=' ')
    
    docs_train = [raw_docs[i] for i in train_index]
    docs_val = [raw_docs[i] for i in val_index]

    # get vocab
    tokenizer = Tokenizer(filters='', lower=False)
    tokenizer.fit_on_texts(docs_train + docs_val)

    num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])
    
    tokenizer = Tokenizer(num_words=num_words, filters='', lower=False)
    tokenizer.fit_on_texts(docs_train + docs_val)    

    docs_train = tokenizer.texts_to_sequences(docs_train)
    docs_val   = tokenizer.texts_to_sequences(docs_val)
    docs_test  = tokenizer.texts_to_sequences(raw_docs_test)

    maxlen = 512
    #     maxlen = max([len(i) for i in  docs_train] + [len(i) for i in  docs_val])
    x_train = pad_sequences(sequences=docs_train, maxlen=maxlen)
    x_val = pad_sequences(sequences=docs_val, maxlen=maxlen)
    x_test = pad_sequences(sequences=docs_test, maxlen=maxlen)

    y_train, y_val = y[train_index], y[val_index]

    input_dim = max(np.max(x_train), np.max(x_val)) + 1
    print('#vocab: {} '.format(num_words), end=' ')
    print(x_train.shape, x_val.shape, x_test.shape)
    

    model = create_model(input_dim)
    
    checkpointer = ModelCheckpoint(filepath='./../fasttext_weights/weights.hdf5', verbose=0, save_best_only=True)

    hist = model.fit(x_train, y_train,
                     batch_size=16,
                     validation_data=(x_val, y_val),
                     epochs=epochs,
                     callbacks=[EarlyStopping(patience=4, monitor='val_loss'), 
                                checkpointer])

    # load best weights
    model.load_weights('./../fasttext_weights/weights.hdf5')
    y_pred = model.predict_proba(x_val)
    sum_loss += log_loss(y_pred=y_pred, y_true=np.nonzero(y_val)[1])
    
    print('valLoss: {}'.format(sum_loss/ite))


    # save features
    predict_prob_features[val_index] = y_pred
    predict_prob_features_test += model.predict_proba(x_test)

1/5: #Trains: 15663, #Val: 3916 #vocab: 28071  (15663, 512) (3916, 512) (8392, 512)
Train on 15663 samples, validate on 3916 samples
Epoch 1/50
15663/15663 [==============================] - 9s - loss: 1.0852 - acc: 0.4036 - val_loss: 1.0803 - val_acc: 0.4035
Epoch 2/50
15663/15663 [==============================] - 10s - loss: 1.0708 - acc: 0.4042 - val_loss: 1.0628 - val_acc: 0.4065
Epoch 3/50
15663/15663 [==============================] - 12s - loss: 1.0385 - acc: 0.4347 - val_loss: 1.0200 - val_acc: 0.4390
Epoch 4/50
15663/15663 [==============================] - 10s - loss: 0.9844 - acc: 0.5377 - val_loss: 0.9649 - val_acc: 0.5143
Epoch 5/50
15663/15663 [==============================] - 11s - loss: 0.9182 - acc: 0.6445 - val_loss: 0.9067 - val_acc: 0.5697
Epoch 6/50
15663/15663 [==============================] - 11s - loss: 0.8502 - acc: 0.7048 - val_loss: 0.8443 - val_acc: 0.6701
Epoch 7/50
15663/15663 [==============================] - 11s - loss: 0.7863 - acc: 0.7389 - val_los

15663/15663 [==============================] - 10s - loss: 0.3100 - acc: 0.9058 - val_loss: 0.4464 - val_acc: 0.8312
Epoch 24/50
15663/15663 [==============================] - 9s - loss: 0.2954 - acc: 0.9120 - val_loss: 0.4385 - val_acc: 0.8292
Epoch 25/50
15663/15663 [==============================] - 10s - loss: 0.2812 - acc: 0.9157 - val_loss: 0.4302 - val_acc: 0.8345
Epoch 26/50
15663/15663 [==============================] - 12s - loss: 0.2678 - acc: 0.9209 - val_loss: 0.4224 - val_acc: 0.8373
Epoch 27/50
15663/15663 [==============================] - 11s - loss: 0.2562 - acc: 0.9255 - val_loss: 0.4204 - val_acc: 0.8373
Epoch 28/50
15663/15663 [==============================] - 10s - loss: 0.2446 - acc: 0.9278 - val_loss: 0.4104 - val_acc: 0.8396
Epoch 29/50
15663/15663 [==============================] - 10s - loss: 0.2342 - acc: 0.9323 - val_loss: 0.4049 - val_acc: 0.8432
Epoch 30/50
15663/15663 [==============================] - 14s - loss: 0.2240 - acc: 0.9350 - val_loss: 0.4051

15664/15664 [==============================] - 8s - loss: 0.1021 - acc: 0.9723 - val_loss: 0.3852 - val_acc: 0.8519
Epoch 50/50
8320/8392 [============================>.] - ETA: 0s

In [6]:
for a, c in author2class.items():
    df['{}_fasttext_unigram'.format(a)] = predict_prob_features[:, c]
    df_test['{}_fasttext_unigram'.format(a)] = predict_prob_features_test[:, c]/num_split

In [11]:
df.to_csv('./../data/train_feature.csv')
df_test.to_csv('./../data/test_feature.csv')